# RSA Demonstration

Import my RSA package into the environment

In [1]:
from RSA import *

### Key Generation
Declare the bit size we want to use

In [2]:
bitSize = 40

Generate a set of RSA keys with appropriate bitsize

In [19]:
RSA_keys = generate_RSA.KeyGen(bitSize, True, True)     # initalise object True sets verbose and demo flag on KeyGen
RSA_keys.generateKeys()                                 # runs generator

Generating  23 bit prime... 
10 rounds of Miller-Rabin needed (according to FISC)
Trial-Division prime list generated, size:  309
New candidate... 5638397
Prime of  23 bits found: 5638397
Checked 1 candidates in 0.00 s
Generating  23 bit prime... 
10 rounds of Miller-Rabin needed (according to FISC)
Trial-Division prime list generated, size:  309
New candidate... 6179183
Prime of  23 bits found: 6179183
Checked 1 candidates in 0.00 s

p: 5638397
q: 6179183

n: 34840686889651
e: 19326399959497
n bit length: 45
Private-Key, d: 6224807444529
Public-Key: (34840686889651, 19326399959497)
Private-Key: 6224807444529
n is 45 bits



True

### Brute Force Algorithm

The simplest approach. 

- Take the $\sqrt{n}$ and check if it divides n. 
- If it does not enumerate each odd smaller than $\sqrt{n}$ until a factor is found. 
- $O(\sqrt{n})$ (ignoring logarithmic test for division)

In [20]:
bf = brute_force.BFSolver(RSA_keys.n, RSA_keys.e, True, True)   # pass it public keys, and True verbose & demo flag
bf.solve()                                                      # run solver

'34840686889651 % 5638397 = 0'

p: 5638397
q: 6179183
Private-Key, d: 6224807444529
Numbers checked: 132102
Space used: 2 bytes
Time taken: 5.146 s


True

### Make it a bit harder

In [5]:
bitSize = 46

RSA_keys = generate_RSA.KeyGen(bitSize, True)     # initalise object True sets verbose flag on KeyGen
RSA_keys.generateKeys()                           # runs generator

Prime of  23 bits found: 7599047
Checked 10 candidates in 0.00 s
Prime of  23 bits found: 5966413
Checked 8 candidates in 0.00 s

p: 7599047
q: 5966413

n: 45339052808411
e: 5216255554453
n bit length: 46
Private-Key, d: 25740546499933
Public-Key: (45339052808411, 5216255554453)
Private-Key: 25740546499933
n is 46 bits



True

### Fermat's Factorisation Method

Relies on Fermat’s observation that any odd number can be represented as the
difference of two squares. 

- We again start with $γ ← \sqrt{n}$
- now we increment γ a single numberat a time until $γ^2 \; mod \; n \; = \; b^2$ where b is an integer. 
- then $\{p,q\} = γ \pm b$
- The worst-case time-complexity is exactlythe same as for brute-force, $O(\sqrt{n})$

In [6]:
ff = fermats.FFSolver(RSA_keys.n, RSA_keys.e, True, True)   # pass it public keys, and True verbose & demo flag
ff.solve()                                                  # run solver

'sqrt(46005426252900 - 45339052808411) = 816317.0'

p: 5966413
q: 7599047
Private-Key, d: 25740546499933
Numbers checked: 49303
Space used: 6 bytes
Time taken: 1.983 s


True

### KNJ - Factorisation

The same as brute-force, but includes a check for prime numbers first. 

- Take the $\sqrt{n}$ and check if it divides n. 
- If it does not enumerate each prime number smaller than $\sqrt{n}$ until a factor is found. 
- $O(\frac{\sqrt{n}}{log(\sqrt{n})})$ (estimation of the number of primes up to $\sqrt{n}$

In [7]:
knj = knj_factorisation.KNJSolver(RSA_keys.n, RSA_keys.e, True, True)   # pass it public keys, and True verbose & demo flag
knj.solve()                                                             # run solver

'45339052808411 % 5966413 = 0'

p: 5966413
q: 7599047
Private-Key, d: 25740546499933
Numbers checked: 48934
Space used: 919314 bytes
Time taken: 1.906 s


True

### Pollard's p-1 

Is our first probabilistic algorithm:

- Utilises Fermat's little theorem 
- for a semi-prime $n \leftarrow pq$, if $x \leftarrow a^{K(p-1)} \equiv 1 \; mod \; p$ then the $gcd(x - 1, n)$ will be divisible by $p$
- We use a B-powersmooth number to find this large multiple
- If our B is too high and both p and q are B-powersmooth it fails
- So is probabilistic and we have to backtrack our B (or increase it)

In [13]:
polMin = pollard_p_minus_1.PSolver(RSA_keys.n, RSA_keys.e, True, True)   # pass it public keys, and True verbose & demo flag
polMin.solve()                                                           # run solver

'Testing B = 1187'

p: 7599047
q: 5966413
Private-Key, d: 25740546499933
Numbers checked: 302
Space used: 0 bytes
Time taken: 0.137 s


True

### Pollard's-$\rho$ method


In [17]:
rho = pollard_rho.RhoSolver(RSA_keys.n, RSA_keys.e, True, True)   # pass it public keys, and True verbose & demo flag
rho.solve()                                                       # run solver

'gcd(13786656544820 - 34631567340637, 45339052808411) = 5966413'

p: 5966413
q: 7599047
Private-Key, d: 25740546499933
Numbers checked: 790
Space used: 10 bytes
Time taken: 0.126 s


True

### Quadratic Sieve

In [6]:
# need to recalculate as only 100% guarenteed for these very small sizes 

bitSize = 20

RSA_keys = generate_RSA.KeyGen(bitSize, True)     # initalise object True sets verbose flag on KeyGen
RSA_keys.generateKeys()                           # runs generator

Prime of  10 bits found: 521
Checked 8 candidates in 0.00 s
Prime of  10 bits found: 853
Checked 1 candidates in 0.00 s

p: 521
q: 853

n: 444413
e: 220771
n bit length: 19
Private-Key, d: 323851
Public-Key: (444413, 220771)
Private-Key: 323851
n is 19 bits



True

In [7]:
quad = quadratic_sieve.QSolver(RSA_keys.n, RSA_keys.e, True, True)  # pass it public keys, and True verbose & demo flag
quad.solve()                                                        # run solver

Length of possible primes: 503
Length of primes with quadRes(n,p) == 1: 258 

p: 521
q: 853
Private-Key, d: 323851
Numbers checked: 13
Space used: 133644 bytes
Time taken: 0.054 s


True

### Basic Analysis
Running the above algorithms on a variety of random inputs to create graphs showing general trends.

The below style code makes the format of the graphs nicer

In [13]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

Runs each algorithm in a thread and plots the results.

- Blue = Brute Force
- Orange = Fermat's Factoring Method
- Green = Pollard's Rho

In [ ]:
%matplotlib notebook
from graphs_RSA import *

minBit = 10
bf_bit = 25
ff_bit = 25
rho_bit = 25

testGraphs(minBit, bf_bit, ff_bit, rho_bit)